In [78]:
import os
import os.path
import random
import numpy as np
import PIL
import cv2
import cv2.xfeatures2d
import matplotlib.pyplot as plt
import keras
import keras.backend as K

In [3]:
def image_path_itr(keyword):
    dir_ = 'downloads/{}/'.format(keyword)
    for f in os.listdir(dir_):
        if not f.endswith('.jpg'):
            continue
        yield dir_ + f

In [34]:
def pil_image_to_numpy(im):
    im = np.array(im.convert('RGB'), dtype=np.uint8)[:,:,::-1]
    return im

In [94]:
def extract_features(keyword):
    surf = cv2.xfeatures2d.SURF_create(10000)
    features = []
    for p in image_path_itr('dragonfly'):
        try:
            im = pil_image_to_numpy(PIL.Image.open(p))
            f = surf.detectAndCompute(im, None)
            if not f[0]:
                continue
            features.append(f)
        except:
            pass
    return features

In [95]:
samples = \
    [(f, 0) for f in extract_features('dragonfly')] + \
    [(f, 1) for f in extract_features('butterfly')]
random.shuffle(samples)
len(samples)

188

In [96]:
train_samples = samples[:-40]
valid_samples = samples[-40:]
train_X = [s[0][1] for s in train_samples]
train_Y = [keras.utils.to_categorical(s[1], num_classes=2) for s in train_samples]
valid_X = [s[0][1] for s in valid_samples]
valid_Y = [keras.utils.to_categorical(s[1], num_classes=2) for s in valid_samples]

In [119]:
K.clear_session()
X = X_input = keras.layers.Input((None, 64))
X = keras.layers.BatchNormalization()(X)
#X = keras.layers.Dropout(0.5)(X)
X = keras.layers.TimeDistributed(keras.layers.Dense(64, activation='relu'))(X)
X = X_desc = keras.layers.TimeDistributed(keras.layers.Dense(64, activation='relu'))(X)
X = keras.layers.TimeDistributed(keras.layers.Dense(1, activation='relu'))(X)
X = keras.layers.Softmax(axis=1)(X)
X = keras.layers.Dot(1)([X, X_desc])
X = keras.layers.Flatten()(X)
X = keras.layers.Dense(2, activation='softmax')(X)
M = keras.models.Model([X_input], [X])
M.compile('adam', loss='categorical_crossentropy', metrics=['accuracy'])
M.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None, 64)     0                                            
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, None, 64)     256         input_1[0][0]                    
__________________________________________________________________________________________________
time_distributed_1 (TimeDistrib (None, None, 64)     4160        batch_normalization_1[0][0]      
__________________________________________________________________________________________________
time_distributed_2 (TimeDistrib (None, None, 64)     4160        time_distributed_1[0][0]         
__________________________________________________________________________________________________
time_distr

In [ ]:
for _ in range(100):
    for X, Y in zip(train_X, train_Y):
        M.fit(
            np.expand_dims(X, axis=0),
            np.expand_dims(Y, axis=0),
            batch_size=1, verbose=False)
    n_correct = 0
    for X, Y in zip(valid_X, valid_Y):
        pred = M.predict(
            np.expand_dims(X, axis=0),
            batch_size=1)[0]
        if np.sum(pred * Y) > 0.5:
            n_correct += 1
    print(n_correct / len(valid_Y))

0.15
0.15
0.175
0.15
0.175
0.225
0.2
0.2
0.2
0.2
0.2
0.2
0.2
0.2
0.2
0.2
0.2
0.175
0.175
0.175
0.175
0.175
0.175
0.175
0.175
0.175
0.175
0.175
0.175
0.175
0.175
0.175
0.175
0.175
0.175
0.175
0.175
0.175
0.175
0.175
0.175
0.175
0.175
0.175
0.175
0.3
0.125
0.15
0.15
0.15
0.15
0.15
0.15
0.125
0.125
0.1
0.15
0.15
0.175
0.175
0.175
0.175
0.175
0.175
0.175
0.175
0.175
0.175
0.175
0.175
0.175
0.175
0.175
0.175
0.175
0.175
0.175
0.175
0.175
0.175
0.175
0.175
0.175
0.175
0.175
0.175
0.175
0.175
0.175
0.175
0.175
0.175
0.175
0.175
